## Análisis para Generar Rutas

### Preprocesamiento

In [50]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
#import numpy as np
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer


###### ArCo ARMADO DE LA BASE#######
import os
from typing import List
from datetime import datetime

# Establecer el directorio de trabajo - DNP
#os.chdir("C:/Users/supegui/Documents/Metodología ArCo/Python/Excel")

# Establecer el directorio de trabajo - Remoto
os.chdir("C:/Backup - Archivos de trabajo/DNP/Automatización ArCo/Excel")


# Saca la lista de archivos del directorio de trabajo
#os.listdir()
# Obtener la fecha actual
fecha_actual = datetime.now().strftime("%Y %m %d")
#print(fecha_actual)

# Construir el nombre del archivo basado en la fecha actual
#nombre_archivo = f"{fecha_actual} Informe ArCo.xlsx"

# Construir el nombre del archivo cuando la fecha no es la actual. Aquí solo se modifica la fecha. 
nombre_archivo = "2024 11 01 - Base Original Desc Completa.xlsx"

data_arco = pd.read_excel(nombre_archivo)

In [51]:
data_arco.head()

,Código,Sector,Entidad,Nombre del Instrumento,Descripción,Descripción larga,Página Web,Antigüedad,Iteración,Emprendedores,...,1. Señale cuál o cuáles de las siguientes brechas,2. ¿En el diseño del instrumento se analizó el posible impacto diferenciado?,3. ¿El instrumento incorpora un enfoque interseccional?,Recursos fuente PGN,Recursos fuente SGR,Recursos fuente Cooperacion,Recursos fuente Esfuerzo Fiscal,Recursos fuente Parafiscales,Recursos fuente Otro,Estado
0,1794,Hacienda,ADMINISTRADORA COLOMBIANA DE PENSIONES - COLPE...,Plan de Educación Financiera,A partir del modelo pedagógico de la entidad p...,A partir del modelo pedagógico de la entidad p...,https://www.colpensiones.gov.co/educacion/,5.0,Segunda iteración 2023 - Mes de reporte: Marzo...,No,...,No,No,No,0,0,0,0,0,0,Desactualizado
1,1800,Hacienda,ADMINISTRADORA COLOMBIANA DE PENSIONES - COLPE...,Programa Beps,Los requisitos para vincularse a Hoy y Mañana ...,Los requisitos para vincularse a Hoy y Mañana ...,https://www.colpensiones.gov.co/beps/,9.0,Segunda iteración 2023 - Mes de reporte: Marzo...,No,...,No,No,No,0,0,0,0,0,0,Desactualizado
2,1797,Hacienda,ADMINISTRADORA COLOMBIANA DE PENSIONES - COLPE...,Silverexpo. Feria de entretenimiento y bienest...,La feria Silverexpo se realiza en alianza con ...,La feria Silverexpo se realiza en alianza con ...,www.colpensiones.gov.co,4.0,Segunda iteración 2023 - Mes de reporte: Marzo...,No,...,No,No,No,0,0,0,0,0,0,Desactualizado
3,3689,Agropecuario,AGENCIA DE DESARROLLO RURAL - ADR,Adecuación de tierras,La adecuación de tierras es el servicio públi...,La adecuación de tierras es el servicio públic...,https://www.adr.gov.co/atencion-y-servicios-a-...,8.0,Primera iteración 2025 – Mes de reporte: Septi...,No,...,Si,No,Si,116422783387,0,0,0,0,0,Actualizado
4,1734,Agropecuario,AGENCIA DE DESARROLLO RURAL - ADR,Fortalecimiento a la prestación del servicio p...,MEJORAR LA TRANSFERENCIA Y APROPIACIÓN DEL CO...,MEJORAR LA TRANSFERENCIA Y APROPIACIÓN DEL CON...,https://www.adr.gov.co/atencion-y-servicios-a-...,4.0,Primera iteración 2024 - Mes de reporte: Septi...,No,...,No,No,No,122305692809,0,0,0,0,0,Desactualizado


In [52]:
# Get data information
data_arco.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 419 entries, 0 to 418
Data columns (total 72 columns):
 #   Column                                                                        Non-Null Count  Dtype         
---  ------                                                                        --------------  -----         
 0   Código                                                                        419 non-null    int64         
 1   Sector                                                                        418 non-null    object        
 2   Entidad                                                                       419 non-null    object        
 3   Nombre del Instrumento                                                        419 non-null    object        
 4   Descripción                                                                   419 non-null    object        
 5   Descripción larga                                                             419 non-null  

In [34]:
# Selecting the relevant features for recommendation - Usuarios completos
selected_features = ['Emprendedores',
                 'Mipymes',
                 'Grandes empresas',
                 'Gobierno ',
                 'Academia',
                 'Entidades de soporte',
                 'Personas naturales',
                 'Compra Pública',
                 'Bonos o Vouchers'
                 ]

print(selected_features)

"""
# Selecting the relevant features for recommendation
selected_features = [
                 #'Emprendedores',
                 'Mipymes',
                 'Grandes empresas',
                 #'Gobierno ',
                 #'Academia',
                 #'Entidades de soporte',
                 #'Personas naturales'
                 ]
print(selected_features)
"""

['Emprendedores', 'Mipymes', 'Grandes empresas', 'Gobierno ', 'Academia', 'Entidades de soporte', 'Personas naturales', 'Compra Pública', 'Bonos o Vouchers']


"\n# Selecting the relevant features for recommendation\nselected_features = [\n                 #'Emprendedores',\n                 'Mipymes',\n                 'Grandes empresas',\n                 #'Gobierno ',\n                 #'Academia',\n                 #'Entidades de soporte',\n                 #'Personas naturales'\n                 ]\nprint(selected_features)\n"

In [36]:
# Replacing the null valuess with null string
for feature in selected_features:
    data_arco[feature] = data_arco[feature].fillna('')

In [38]:
# combining all the 5 selected features - Con todos los tipos de usuarios
combined_features = data_arco['Emprendedores'] + ' ' + data_arco['Mipymes'] + ' ' + data_arco['Grandes empresas'] + ' ' + data_arco['Gobierno '] + ' ' + data_arco['Academia'] + ' ' + data_arco['Entidades de soporte']+ ' ' + data_arco['Personas naturales'] + ' ' + data_arco['Compra Pública'] + ' ' + data_arco['Bonos o Vouchers']
combined_features



0      No No No No No No No No No
1      No No No No No No No No No
2      No No No No No No No No No
3      No No No No No No Si Si No
4      No No No No No No Si No No
                  ...            
414    No No No Si Si Si Si No No
415    No No Si Si Si Si Si Si No
416    No No No Si Si Si Si No No
417    No Si Si Si No Si No Si No
418    No No No No No No Si No No
Length: 419, dtype: object

### Construyendo el sistema de recomendación con el nombre del instrumento

In [39]:
# converting the text data to feature vectors
vectorizer = TfidfVectorizer()

feature_vectors = vectorizer.fit_transform(combined_features)

In [40]:
print(feature_vectors)

  (0, 0)	1.0
  (1, 0)	1.0
  (2, 0)	1.0
  (3, 1)	0.27836971610212
  (3, 0)	0.9604739981682092
  (4, 1)	0.12579139477659648
  (4, 0)	0.9920567146086752
  (5, 1)	0.4523392459085436
  (5, 0)	0.8918459545296429
  (6, 1)	0.12579139477659648
  (6, 0)	0.9920567146086752
  (7, 1)	0.4523392459085436
  (7, 0)	0.8918459545296429
  (8, 1)	0.4523392459085436
  (8, 0)	0.8918459545296429
  (9, 1)	0.4523392459085436
  (9, 0)	0.8918459545296429
  (10, 1)	0.27836971610212
  (10, 0)	0.9604739981682092
  (11, 1)	0.4523392459085436
  (11, 0)	0.8918459545296429
  (12, 1)	0.27836971610212
  (12, 0)	0.9604739981682092
  (13, 1)	0.27836971610212
  (13, 0)	0.9604739981682092
  :	:
  (406, 1)	0.12579139477659648
  (406, 0)	0.9920567146086752
  (407, 1)	0.9924929508920971
  (407, 0)	0.12230184965689293
  (408, 0)	1.0
  (409, 1)	0.27836971610212
  (409, 0)	0.9604739981682092
  (410, 1)	0.6301299324623892
  (410, 0)	0.7764897090206313
  (411, 1)	0.9625476282805203
  (411, 0)	0.27111263949057307
  (412, 1)	0.78519633

In [41]:
# getting the similarity scores using cosine similarity
similarity = cosine_similarity(feature_vectors, feature_vectors)

In [42]:
print(similarity)

[[1.         1.         1.         ... 0.77648971 0.61924689 0.99205671]
 [1.         1.         1.         ... 0.77648971 0.61924689 0.99205671]
 [1.         1.         1.         ... 0.77648971 0.61924689 0.99205671]
 ...
 [0.77648971 0.77648971 0.77648971 ... 1.         0.97561455 0.84958675]
 [0.61924689 0.61924689 0.61924689 ... 0.97561455 1.         0.71309898]
 [0.99205671 0.99205671 0.99205671 ... 0.84958675 0.71309898 1.        ]]


In [43]:
# creating a list with all the movie names given in the dataset

list_of_all_titles = data_arco['Nombre del Instrumento'].tolist()
print(list_of_all_titles)

['Plan de Educación Financiera', 'Programa Beps', 'Silverexpo. Feria de entretenimiento y bienestar para Personas Mayores', 'Adecuación de tierras', 'Fortalecimiento a la prestación del servicio público de extensión Agropecuaria', 'FORTALECIMIENTO DE LA GESTIÓN Y APROPIACIÓN DEL CONOCIMIENTO TÉCNICO DE  LOS PROCESOS PRODUCTIVOS AGROPECUARIOS Y RURALES, EN LOS  PRODUCTORES Y LAS A', 'FORTALECIMIENTO DE LA GESTIÓN Y APROPIACIÓN DEL CONOCIMIENTO TÉCNICO DE LOS PROCESOS PRODUCTIVOS AGROPECUARIOS Y RURALES', 'IMPLEMENTACIÓN DE ESTRATEGIAS DE FORTALECIMIENTO Y APOYO COMERCIAL PARA ORGANIZACIONES RURALES A NIVEL NACIONAL', 'Proyectos Integrales de Desarrollo Agropecuario y Rural (PIDAR)', 'Obras por Impuestos', 'Asistencias técnicas realizadas', 'Escuela Virtual de Seguridad Vial', 'ORIENTACIONES TÉCNICAS PARA LA ELABORACIÓN DE PLANES LOCALES DE SEGURIDAD VIAL', 'Red de Observatorios Territoriales de Seguridad Vial (ROT)', 'SEGUIMIENTO A LA IMPLEMENTACIÓN DE LA RUTA DE ATENCIÓN INTEGRAL A VIC

In [30]:
# getting the movie name from the user
instrument_name = input(' Introducir el nombre del instrumento : ')
#instrument_name = 'Información estadística de la temática económica'


In [31]:
# finding the close match for the instrument name given by the user
find_close_match = difflib.get_close_matches(instrument_name, list_of_all_titles)
print(find_close_match)

['Fondo Emprender', 'Emprendetón']


In [14]:
from difflib import SequenceMatcher

def calculate_similarity(a, b):
    return SequenceMatcher(None, a, b).ratio()

# Ordenar los instrumentos según similitud con el nombre escogido
instrumentos_similares = sorted(
    list_of_all_titles,
    key=lambda x: calculate_similarity(instrument_name, x),
    reverse=True
)[:10]


In [15]:
print(instrumentos_similares)

['Fábricas de Productividad y Sostenibilidad', 'Fábricas de Internacionalización', 'Categorías de ruralidad', 'Ciber Paz – Sensibilizaciones', 'Materiales de Referencia Certificados', 'Convocatorias de Tesis y Pasantias', 'Programa de Fortalecimiento Productivo y Comercial a Pueblos Étnicos', 'Hábitos y Estilos de Vida Saludable', 'Arrendamiento sociales de activos', 'Sistema de Información sobre Biodiversidad de Colombia - SiB Colombia']


### Análisis a partir de descripciones y features

In [53]:
# Fill missing values with empty strings
data_arco['Descripción larga'] = data_arco['Descripción larga'].fillna('')

# Optionally, convert text to lowercase
data_arco['Descripción larga'] = data_arco['Descripción larga'].str.lower()

In [110]:
phrases_to_exclude = [
    "Certificaciones, acreditaciones, etc",
    "Encadenamientos productivos, rutas competitivas",
    "Facilitación del comercio al interior del país y en el extranjero",
    "Adopción Tecnológica",
    "Página web, tienda virtual, acceso a Marketplace",
    "Habilidades Digitales para Comercio electrónico",
    "Cursos, formación técnica, charlas",
    "Gestión de Pagos",
    "Acceso a pasarelas de pagos sistemas de pago electrónicos, programas de inclusión financiera. Bancarización",
    "Logística",
    "Acceso a servicios de logística",
    "Marco Normativo y Regulatorio",
    "Cursos, capacitación, charlas, informativos",
    "Habilidades y competencias para el uso de tecnologías e Internet que facilitan y fomentan el uso efectivo de datos, dispositivos digitales, tecnologías y servicios digitales",
    "Bioeconomía, Energías Renovables, Eficiencia Energética, Movilidad Eléctrica, Agricultura Sostenible, Economía Circular",
    "Reconocimiento e impulso a la Economía Popular",
    "Reconocimiento, visibilización y fortalecimiento de las unidades productivas de la economía popular entendida como los oficios y ocupaciones mercantiles (producción, distribución y comercialización de bienes y servicios) y no mercantiles (domésticas o comunitarias) desarrolladas por unidades económicas de baja escala (personales, familiares, micronegocios o microempresas), en cualquier sector económico.",
    "Enseñar conceptos de economía",
    "Planear y ahorrar en las diferentes etapas de la vida",
    "Enseñar sobre inversiones",
    "Usar responsablemente el crédito",
    "Prevenir el fraude o afectación del patrimonio",
    "Aumentar el uso de pagos digitales/electrónicos",
    "Incrementar la formalización y/o uso de productos financieros",
    "Educar en temas tributarios",
    "Fomentar el emprendimiento",
    "Fomentar la planeación empresarial",
    "Informar sobre la protección al consumidor financiero",
    "Gestionar riesgos y promover la utilización de seguros",
    "No hay objetivo particular",
    "Cuenta con otros objetivos",
    "Educar en temas tributarios",
    "Fomentar el emprendimiento",
    "Fomentar la planeación empresarial",
    "Informar sobre la protección al consumidor financiero",
    "Gestionar riesgos y promover la utilización de seguros",
    "No hay objetivo particular",
    "Cuenta con otros objetivos",
    "El instrumento se dirige a la creación de nuevas empresa o intraemprendimiento.",
    "El instrumento se dirige a la creación y/o fortalecimiento de emprendimientos de base científico- tecnológica.",
    "Desarrollo y fomento de instrumentos financieros",
    "Cursos",
    "Actividades académicas cortas presenciales o virtuales",
    "Talleres o Entrenamientos",
    "Capacitaciones específicas orientadas a la práctica",
    "Charlas",
    "Conferencias",
    "Pasantías o Estancias",
    "Acompañamiento práctico o ejercicio profesional",
    "Básica y media",
    "Preescolar, primaria y secundaría",
    "Técnica o Tecnológica",
    "Preescolar, primaria y secundaría",
    "Profesional",
    "Formación Universitaria",
    "Posgrado",
    "Especialización, maestría, doctorado",
    "Registro y cumplimiento de requisitos legales en empresas y actividades laborales",
    "Acceso y uso de productos financieros desarrollados a la medida de las necesidades de sus usuarios",
    "Productos o procesos nuevos o mejorados",
    "Ampliación del conocimiento a nivel teórico o práctico",
    "Protección y/o registro de derechos de Propiedad Intelectual",
    "Apoyo o acompañamiento para la solicitud o registro de derechos de propiedad intelectual como patentes, diseños industriales, marcas, registros de software, derechos de autor y derechos conexos, derechos de obtentor de variedades vegetales u otros instrumentos de protección.",
    "Gestión y comercialización de derechos de propiedad intelectual",
    "Apoyo o acompañamiento para transferir o comercializar derechos de propiedad intelectual, por ejemplo a través de mecanismos como licenciamiento y cesión de derechos, entre otros.",
    "Observancia de los derechos de propiedad intelectual",
    "Actividades relacionadas con sensibilización, identificación y reducción de las infacciones a los derechos de propiedad intelectual, procesos jurisdiccionales, de investigación de delitos, servicios de resolución de conflictos, arbitraje y otros encaminados a la defensa efectiva de los derechos de propiedad intelectual.",
    "Difusión y sensibilización sobre derechos de propiedad intelectual",
    "Actividades dirigidas a fomentar el conocimiento, formación y apropiación alrededor de la propiedad intelectual para promover la cultura de respeto por los derechos y visibilizar sus beneficios.",
    "Procesos destinados a compartir ideas, conocimientos, tecnologías y capacidades",
    "Transferencia de tecnología y conocimiento: Proceso de difusión de tecnologías y conocimientos, desde el actor o actores fuente de su generación, hacia el actor o actores que buscan apropiarlos para darles aplicación en la industria, la sociedad o el mercado. Algunos mecanismos de transferencia incluyen el licenciamiento y la cesión de derechos de propiedad intelectual, la creación de empresas de base científico tecnológica, la cooperación tecnológica, entre otras.",
    "X - Si el instrumento cuenta con Enfoque de Género por favor marque aquí!", 
    "Si el instrumento esta dirigido por enfoque de género, seleccione esta opción"
    "X -Si el instrumento es de educación financiera por favor marque aquí!",
    "Si el instrumento es de educación financiera por favor cuéntenos ¿qué beneficios se han podido identificar en la población?",
    "X -Si el instrumento es de emprendimiento por favor marque aquí!",
    "X -Si el instrumento es de propiedad intelectual por favor marque aquí!",
    "Si el objetivo es de propiedad intelectual por favor cuéntenos ¿qué tipos de derechos de propiedad intelectual contempla?",
    "Acción por el clima",
    "Adoptar medidas urgentes para combatir el cambio climático y sus efectos.",
    "Agua limpia y saneamiento",
    "Garantizar la disponibilidad de agua y su gestión sostenible y el saneamiento para todos.",
    "Alianzas para lograr los objetivos",
    "Fortalecer los medios de ejecución y revitalizar la Alianza Mundial para el Desarrollo Sostenible.",
    "Ciudades y comunidades sostenibles",
    "Lograr que las ciudades y los asentamientos humanos sean inclusivos, seguros, resilientes y sostenibles.",
    "Educación de calidad",
    "Garantizar una educación inclusiva, equitativa y de calidad y promover oportunidades de aprendizaje durante toda la vida para todos.",
    "Energía asequible y no contaminante",
    "Garantizar el acceso a una energía asequible, segura, sostenible y moderna para todos.",
    "Fin de la pobreza",
    "Poner fin a la pobreza en todas sus formas en todo el mundo.",
    "Hambre cero",
    "Poner fin al hambre, lograr la seguridad alimentaria y la mejora de la nutrición y promover la agricultura sostenible.",
    "Igualdad de género",
    "Lograr la igualdad entre los géneros y empoderar a todas las mujeres y las niñas.",
    "Industria, innovación e infraestructura",
    "Construir infraestructuras resilientes, promover la industrialización inclusiva y sostenible y fomentar la innovación.",
    "Paz, justicia e instituciones sólidas",
    "Promover sociedades pacíficas e inclusivas para el desarrollo sostenible, facilitar el acceso a la justicia para todos y crear instituciones eficaces, responsables e inclusivas a todos los niveles.",
    "Producción y consumo responsables",
    "Garantizar modalidades de consumo y producción sostenibles.",
    "Reducción de las desigualdades",
    "Reducir la desigualdad en y entre los países.",
    "Salud y bienestar",
    "Garantizar una vida sana y promover el bienestar para todos en todas las edades.",
    "Trabajo decente y crecimiento económico",
    "Promover el crecimiento económico sostenido, inclusivo y sostenible, el empleo pleno y productivo y el trabajo decente para todos.",
    "Vida de ecosistemas terrestres",
    "Proteger, restablecer y promover el uso sostenible de los ecosistemas terrestres, gestionar los bosques de forma sostenible, luchar contra la desertificación, detener e invertir la degradación de las tierras y poner freno a la pérdida de la diversidad biológica.",
    "Vida submarina",
    "Conservar y utilizar en forma sostenible los océanos, los mares y los recursos marinos para el desarrollo sostenible.",
    "Actividades artísticas, de entretenimiento y recreación",
    "Actividades de atención de la salud humana y de asistencia social",
    "Actividades de los hogares individuales en calidad de empleadores; actividades no diferenciadas de los hogares individuales como productores de bienes y servicios para uso propio.",
    "Actividades de organizaciones y entidades extraterritoriales",
    "Actividades de servicios administrativos y de apoyo",
    "Actividades financieras y de seguros",
    "Actividades inmobiliarias",
    "Actividades profesionales, científicas y técnicas",
    "Administración pública y defensa; planes de seguridad social de afiliación obligatoria",
    "Agricultura, ganadería, caza, silvicultura y pesca",
    "Alojamiento y servicios de comida",
    "Comercio al por mayor y al por menor; reparación de vehículos automotores y motocicletas",
    "Construcción",
    "Distribución de agua; evacuación y tratamiento de aguas residuales, gestión de desechos y actividades de saneamiento ambiental",
    "Educación",
    "Explotación de minas y canteras",
    "Industrias manufactureras",
    "Información y comunicaciones",
    "Otras actividades de servicios",
    "Suministro de electricidad, gas, vapor y aire acondicionado",
    "Transporte y almacenamiento",
    "Apoyo Financiero",
    "Dinero para invertir en en organizaciones, proyectos o personas naturales",
    "Asistencia técnica y Consultoría",
    "Servicios de mentoría, consultoría, entre otras",
    "Bonos o Vouchers",
    "Cupones con pequeñas subvenciones para adquirir servicios de proveedores de conocimiento externos",
    "Compra Pública",
    "Compra de equipos tecnológicos y maquinaria para crear o mejorar productos y servicios",
    "Eventos",
    "Espacio para la promoción de organizaciones, proyectos, programas, etc.",
    "Formación de Talento Humano",
    "Procesos educativos para el desarrollo o fortalecimiento de habilidades",
    "Incentivos Tributarios",
    "Deducciones, exenciones por inversiones para la declaración de gastos",
    "Infraestructura",
    "Provisión de infraestructura para fomentar la innovación",
    "Instrumentos Regulatorios",
    "Reglamentación por parte de las autoridades públicas y organismos gubernamentales",
    "Premios y Reconocimientos",
    "Recompensas monetarias, en especie o simbólicas por desempeño o cualidades",
    "Redes de Colaboración",
    "Conexiones formales o informales de relacionamiento o trabajo conjunto",
    "Sistemas de Información",
    "Administrar, recolectar, procesar, almacenar y distribuir información",
    "Abejorral", "Abriaquí", "Alejandría", "Amalfi", "Angostura", "Anorí", "Apartadó", 
    "Argelia", "Briceño (Antioquia)", "Cáceres", "Caicedo", "Campamento", "Carepa", "Caucasia", 
    "Chigorodó", "Cocorná", "Concepción (Antioquia)", "Dabeiba", "El Bagre", "Frontino", 
    "Granada (Antioquia)", "Guadalupe (Antioquia)", "Ituango", "Montebello", "Murindó", "Mutata", 
    "Nariño (Antioquia)", "Nechí", "Necoclí", "Puerto Berrio", "Remedios", "Salgar", "San Andrés De Cuerquia", 
    "San Carlos (Antioquia)", "San Francisco (Antioquia)", "San Luis (Antioquia)", "San Pedro De Uraba", "San Rafael", 
    "Santafé De Antioquia", "Santo Domingo", "Segovia", "Sonson", "Tarazá", "Toledo (Antioquia)", "Turbo", "Uramita", 
    "Urrao", "Valdivia", "Vegachí", "Vigía Del Fuerte", "Yalí", "Yarumal", "Yolombó", "Yondó", "Zaragoza", "Arauca", 
    "Arauquita", "Cravo Norte", "Fortul", "Puerto Rondón", "Saravena", "Tame", "Arenal", "Cantagallo", "Cordoba (Bolivar)", 
    "El Carmen De Bolivar", "El Guamo", "Maria La Baja", "Montecristo", "Morales (Bolivar)", "Río Viejo", "San Jacinto", 
    "San Juan Nepomuceno", "San Pablo (Bolivar)", "Santa Rosa (Bolivar)", "Simití", "Tiquisio", "Zambrano", "Chivor", 
    "Labranzagrande", "Pajarito", "Paya", "Pisba", "Anserma", "Belalcázar", "Marulanda", "Norcasia", "Palestina (Caldas)", 
    "Pensilvania", "Riosucio (Caldas)", "Samaná", "Albania (Caquetá)", "Belén De Los Andaquies", "Cartagena Del Chairá", 
    "Curillo", "El Doncello", "El Paujil", "Florencia (Caquetá)", "La Montañita", "Milan", "Morelia", "Puerto Rico (Caquetá)", 
    "San Jose Del Fragua", "San Vicente Del Caguán", "Solano", "Solita", "Valparaiso (Caquetá)", "Aguazul", "Chameza", "Hato Corozal", 
    "La Salina", "Maní", "Monterrey", "Paz De Ariporo", "Pore", "Recetor", "Sabanalarga (Casanare)", "Sácama", "Támara", "Tauramena", "Villanueva (Casanare)",
    "Alcaldías", "Empresas Públicas", "Gobernaciones", "Grandes empresas", "Mediana Empresa", "Microempresa", "Ministerios", "Ingeniería y Tecnología",
    "Comunidades",  "Pequeña Empresa",  "Asalariados", "Bachilleres graduados", "Centros educativos", "Comisiones Regionales de Competitividad e Innovación (CRCI)",  
    "Contralorías", "Defensa", "Departamentos", "Administrativos", "Direcciones", "Distrito Capital", "Docentes formadores instructores", "Embajadas de Colombia", 
    "Embajadas en Colombia",  "Entidades Gubernamentales varias", "Estudiantes de colegio", "Estudiantes de formación para el trabajo y desarrollo humano", "Estudiantes de postgrado", 
    "Estudiantes de pregrado", "Fondos",  "Hogares/Familias", "Instituciones educativas", "Investigadores", "Jóvenes",  "Minorías étnicas", "Mujeres Organismos", "Internacionales", "Población rural", "Superintendencias", "Todas las edades", "Ciencias Médicas y de Salud", "Ciencias Sociales",
    "Docentes", "formadores", "instructores", "Grupos de investigación, desarrollo tecnológico e innovación", "Humanidades",
    "Instituciones universitarias o escuelas tecnológicas", "Universidades",
    "victimas", "Salud mental", "Ruta de atención", "Responsabilidad civil", "Atención integral victima siniestro vial", "Atención a victimas de siniestros viales", "Cámaras de comercio", "Empleados", "Emprendedores", "Entre 12 y 17 años (adolescentes)", "Entre 18 y 28 años (adultos jóvenes)", 
    "Entre 18 y 55 años (mayores de edad en etapa activa)", "Entre 5 y 11 años (niños)", "Estudiantes técnicos o tecnológicos", "Fundaciones", "Gremios", "Instituciones técnicas profesionales", "Instituciones tecnológicas", "Madres cabeza de familia", "Mayores a 56 años", "Migrantes", "Mujeres", 
    "No está segmentado por edades", "ONGs", "Organismos", "Organizaciones del sector productivo", "Personas en condición de discapacidad", "Personas sobre endeudadas", "Población de la tercera edad", "Población no bancarizada", "Población vulnerable", "es con título de doctorado", "es con título de maestría", 
    "es técnicos", "es tecnólogos", "es universitarios", "Público general bancarizado", "Solo a desempleados", "Solo a pensionados", "Trabajadores independientes", "Unidades de la Economía Popular", "Víctimas del conflicto armado",
    "Ciencias Naturales", "Formalización", "formalizacion", "Ciencias Agrícolas", "Cooperativas", "Semilleros de investigación", "conocimiento", "capacidades", "cooperación", "Aceleradoras", "Asociaciones Ad Hoc", "Centros de desarrollo empresarial", "Centros de Desarrollo Infantil", "Empresas del sector solidario", "Empresas en etapa de consolidación", 
    "Empresas en etapa de creación", "Empresas en etapa de crecimiento", "de inversión", "Grupos locales de I+D+I", "Incubadoras", "Instituciones científicas y/o tecnológicas privadas sin fines de lucro", "Receptores de remesas", "Uniones temporales", "convocatoria", "financiación",
    "Encadenamientos productivos", "rutas de competitivas", "pesca", "acuicultura", "elearning", "cursos virtuales", "pedagogía digital", "cursos interactivos", "cursos lúdicos", "formación ambiental virtual", "cursos virtuales ambientales", "cursos licenciamiento ambiental", "educación online", "multimedia", "interactivo", "lúdico", "objetivos virtuales de aprendizaje", 
    "lsm", "plataforma educativa", "diseño instrucional", "ambientales", "Modelación", "Monitoreo", "Analitica de datos", "pronostico", "impacto ambiental", "tableros de control", "Acceso a la información",
    "Autoridad Nacional de Licencias Ambientales", "conflictos medio ambiente", "Conflictos Sociales", "Conflictos Ambientales", "Conflictos Socioecológicos", "Conflictividad", "recursos naturales", "Acciones Institucionales", "Actores Territoriales", "Protesta Social", "Acuerdo de Escazú",
    "PROYECTOS PRODUCTIVOS", "credito", "financiamiento", "Joven", "actividades rurales", 
    "Mujer Rural",  "actividad agropecuaria",  "mujer", "economía campesina", "cultivo", "Formación en calidad", "Asistencia técnica empresarial", 
    "ADMINISTRADORA COLOMBIANA DE PENSIONES - COLPENSIONES",
    "AGENCIA DE DESARROLLO RURAL - ADR",
    "AGENCIA DE RENOVACION DEL TERRITORIO – ART",
    "AGENCIA NACIONAL DE MINERÍA - ANM",
    "AGENCIA NACIONAL DE SEGURIDAD VIAL",
    "AGENCIA NACIONAL DE TIERRAS - ANT",
    "AGENCIA NACIONAL DEL ESPECTRO - ANE",
    "AGENCIA PARA LA REINCORPORACION Y LA NORMALIZACION - ARN",
    "AGENCIA PRESIDENCIAL DE COOPERACIÓN INTERNACIONAL DE COLOMBIA, APC - COLOMBIA",
    "AUTORIDAD NACIONAL DE ACUICULTURA Y PESCA - AUNAP",
    "AUTORIDAD NACIONAL DE LICENCIAS AMBIENTALES ANLA",
    "BANCO AGRARIO  DE COLOMBIA",
    "BANCO COLOMBIANO PARA EL DESARROLLO - BANCOLDEX",
    "Bancóldex S.A.",
    "COLOMBIA PRODUCTIVA",
    "COMISIÓN DE REGULACIÓN DE COMUNICACIONES (CRC)",
    "CORPORACION AUTONOMA REGIONAL DEL RIO GRANDE DE LA MAGDALENA - CORMAGDALENA",
    "CORPORACIÓN COLOMBIANA DE INVESTIGACIÓN AGROPECUARIA - AGROSAVIA",
    "CORPORACION PARA EL DESARROLLO SOSTENIBLE DEL AREA DE MANEJO ESPECIAL LA MACARENA - CORMACARENA",
    "DEPARTAMENTO ADMINISTRATIVO DE LA FUNCION PUBLICA",
    "DEPARTAMENTO ADMINISTRATIVO DE LA PRESIDENCIA DE LA REPÚBLICA - DAPRE",
    "DEPARTAMENTO ADMINISTRATIVO NACIONAL DE ESTADISTICA (DANE)",
    "DEPARTAMENTO ADMINISTRATIVO NACIONAL DE PLANEACION",
    "Departamento Nacional de Planeación - DNP",
    "DIRECCION DE IMPUESTOS Y ADUANAS NACIONALES (DIAN)",
    "DIRECCION NACIONAL DEL DERECHO DE AUTOR",
    "FINAGRO - Fondo para el financiamiento del sector agropecuario",
    "FONDO ADAPTACION",
    "Fondo Mujer Emprende",
    "FONDO NACIONAL DE GARANTÍAS (FNG)",
    "FONDO NACIONAL DE TURISMO - FONTUR",
    "Fondo Nacional del Ahorro - FNA",
    "ICETEX",
    "INNPULSA COLOMBIA",
    "INSTITUTO AMAZÓNICO DE INVESTIGACIONES CIENTÍFICAS SINCHI",
    "INSTITUTO CARO Y CUERVO",
    "INSTITUTO COLOMBIANO AGROPECUARIO",
    "INSTITUTO COLOMBIANO DE BIENESTAR FAMILIAR (ICBF)",
    "INSTITUTO DE HIDROLOGIA, METEOROLOGIA Y ESTUDIOS AMBIENTALES- IDEAM",
    "INSTITUTO DE INVESTIGACIONES MARINAS Y COSTERAS -INVEMAR",
    "INSTITUTO DE PLANIFICACION Y PROMOCION DE SOLUCIONES  ENERGETICAS PARA LAS ZONAS NO INTERCONECTADAS -IPSE-",
    "INSTITUTO GEOGRAFICO AGUSTIN CODAZZI - IGAC",
    "INSTITUTO HUMBOLDT",
    "INSTITUTO NACIONAL DE CANCEROLOGIA",
    "INSTITUTO NACIONAL DE METROLOGÍA - INM",
    "INSTITUTO NACIONAL DE SALUD (INS)",
    "INSTITUTO NACIONAL DE VIGILANCIA DE MEDICAMENTOS Y ALIMENTOS - INVIMA",
    "INSTITUTO NACIONAL PARA CIEGOS (INCI)",
    "INSTITUTO NACIONAL PARA SORDOS (INSOR)",
    "INSTITUTO NACIONAL PENITENCIARIO Y CARCELARIO - INPEC",
    "JURISDICCION ESPECIAL PARA LA PAZ",
    "MINISTERIO DE AGRICULTURA Y DESARROLLO RURAL",
    "MINISTERIO DE AMBIENTE Y DESARROLLO SOSTENIBLE",
    "Ministerio de Ciencia, Tecnología e Innovación",
    "MINISTERIO DE COMERCIO, INDUSTRIA Y TURISMO",
    "MINISTERIO DE CULTURA",
    "Ministerio de Deporte",
    "MINISTERIO DE EDUCACION NACIONAL",
    "MINISTERIO DE HACIENDA Y CREDITO PUBLICO",
    "MINISTERIO DE JUSTICIA Y DEL DERECHO",
    "MINISTERIO DE MINAS Y ENERGIA",
    "MINISTERIO DE RELACIONES EXTERIORES",
    "MINISTERIO DE SALUD Y PROTECCION SOCIAL",
    "MINISTERIO DE TECNOLOGIAS DE LA INFORMACION Y LAS COMUNICACIONES",
    "MINISTERIO DE TRANSPORTE",
    "MINISTERIO DEL TRABAJO",
    "PARQUES NACIONALES NATURALES DE COLOMBIA",
    "Positiva Compañía de Seguros",
    "PRESIDENCIA DE LA REPUBLICA",
    "PROCOLOMBIA - EXPORTACIONES, TURISMO, INVERSIÓN, MARCA PAÍS",
    "PROIMÁGENES COLOMBIA",
    "SERVICIO GEOLÓGICO COLOMBIANO",
    "SERVICIO NACIONAL DE APRENDIZAJE - SENA",
    "SOCIEDAD DE ACTIVOS ESPECIALES S.A.S",
    "SUPERINTENDENCIA DE INDUSTRIA Y COMERCIO",
    "SUPERINTENDENCIA DE PUERTOS Y TRANSPORTE",
    "SUPERINTENDENCIA DE SOCIEDADES",
    "SUPERINTENDENCIA FINANCIERA DE COLOMBIA",
    "SUPERINTENDENCIA NACIONAL DE SALUD",
    "UNIDAD ADMINISTRATIVA ESPECIAL  - AGENCIA NACIONAL DE CONTRATACIÓN PÚBLICA - COLOMBIA COMPRA EFICIENTE",
    "UNIDAD ADMINISTRATIVA ESPECIAL AGENCIA NACIONAL DE DEFENSA JURIDICA DEL ESTADO",
    "UNIDAD ADMINISTRATIVA ESPECIAL DE GESTIÓN DE RESTITUCIÓN DE TIERRAS DESPOJADAS",
    "UNIDAD ADMINISTRATIVA ESPECIAL DE LA AERONAUTICA CIVIL",
    "UNIDAD ADMINISTRATIVA ESPECIAL DE ORGANIZACIONES SOLIDARIAS - UAEOS",
    "UNIDAD ADMINISTRATIVA ESPECIAL DEL SERVICIO PUBLICO DE EMPLEO",
    "UNIDAD DE BUSQUEDA DE PERSONAS DADAS POR DESAPARECIDAS EN EL CONTEXTO Y EN RAZÓN DEL CONFLICTO ARMADO -UBDP",
    "UNIDAD DE INFORMACION Y ANALISIS FINANCIERO",
    "UNIDAD DE PLANEACIÓN MINERO – ENERGÉTICA UPME",
    "UNIDAD DE PLANEACION MINERO ENERGETICA - UPME",
    "UNIDAD PARA LA ATENCIÓN Y REPARACIÓN INTEGRAL A LAS VICTIMAS",
    "X -Si el instrumento es de educación financiera por favor marque aquí!",
    "Alcaldía de Manizales"                                           
]

import re

# Function to remove phrases
def remove_phrases(text):
    for phrase in phrases_to_exclude:
        # Create a regex pattern for the phrase with optional punctuation
        pattern = re.escape(phrase) + r'[\.,;"]*'
        text = re.sub(pattern, '', text)
    return text.strip()

# Apply the function to the 'Descripción larga' column
data_arco['Descripción larga'] = data_arco['Descripción larga'].apply(remove_phrases)

# Display the updated descriptions
print(data_arco['Descripción larga'].head())

0    a partir del modelo pedagógico de la entidad p...
1    los requisitos para vincularse a hoy y mañana ...
2    la feria silverexpo se realiza en alianza con ...
3    la adecuación de tierras es el servicio públic...
4    mejorar la transferencia y apropiación del  en...
Name: Descripción larga, dtype: object


In [64]:
# Fill missing values with empty strings
data_arco['Descripción larga'] = data_arco['Descripción larga'].fillna('')

# Convert text to lowercase
data_arco['Descripción larga'] = data_arco['Descripción larga'].str.lower()

# Vectorize the descriptions
vectorizer_desc = TfidfVectorizer()
description_vectors = vectorizer_desc.fit_transform(data_arco['Descripción larga'])

In [104]:
# Selected features
selected_features = ['Emprendedores', 'Mipymes', 'Grandes empresas', 'Gobierno ', 'Academia', 'Entidades de soporte', 'Personas naturales', 'Bonos o Vouchers', 'Compra Pública']

# Fill missing values with empty strings
for feature in selected_features:
    data_arco[feature] = data_arco[feature].fillna('')

# Combine the features into a single string
combined_features = data_arco['Emprendedores'] + ' ' + data_arco['Mipymes'] + ' ' + data_arco['Grandes empresas'] + ' ' + data_arco['Gobierno '] + ' ' + data_arco['Academia'] + ' ' + data_arco['Entidades de soporte'] + ' ' + data_arco['Personas naturales'] + ' ' + data_arco['Bonos o Vouchers'] + ' ' + data_arco['Compra Pública']

# Vectorize the combined features
vectorizer_features = TfidfVectorizer()
feature_vectors = vectorizer_features.fit_transform(combined_features)

In [105]:
from sklearn.metrics.pairwise import cosine_similarity

# Description similarity matrix
description_similarity = cosine_similarity(description_vectors, description_vectors)

# Features similarity matrix
features_similarity = cosine_similarity(feature_vectors, feature_vectors)

In [106]:
# Define weights
weight_desc = 0.7
weight_features = 0.3

# Ensure both matrices are in the range [0, 1]
# If necessary, normalize the matrices
# For cosine similarity, values are already between -1 and 1, but we can adjust them to [0, 1]
description_similarity = (description_similarity + 1) / 2
features_similarity = (features_similarity + 1) / 2

# Combine the similarity matrices
combined_sim = (weight_desc * description_similarity) + (weight_features * features_similarity)

In [107]:
from difflib import SequenceMatcher

def find_closest_name(query, names):
    max_ratio = 0
    closest_name = ""
    for name in names:
        ratio = SequenceMatcher(None, query, name).ratio()
        if ratio > max_ratio:
            max_ratio = ratio
            closest_name = name
    return closest_name

def recommend_instruments(input_name, data_arco, combined_sim, top_n=5):
    # Find the closest instrument name
    closest_name = find_closest_name(input_name, data_arco['Nombre del Instrumento'])
    # Get the index of the selected instrument
    selected_index = data_arco[data_arco['Nombre del Instrumento'] == closest_name].index[0]
    # Get similarity scores
    sim_scores = list(enumerate(combined_sim[selected_index]))
    # Sort the scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    # Exclude the selected instrument
    sim_scores = [x for x in sim_scores if x[0] != selected_index]
    # Get top N indices
    top_indices = [i[0] for i in sim_scores[:top_n]]
    # Return recommended instruments
    return data_arco.iloc[top_indices][['Código', 'Nombre del Instrumento', 'Sector', 'Entidad']]

In [109]:
# Example usage
recommendations = recommend_instruments("Emprendetón", data_arco, combined_sim, top_n=20)
print(recommendations)

     Código                             Nombre del Instrumento  \
146    1758                                        Héroes Fest   
51     2420                                        Jover Rural   
35     3570  Beneficio de Capital Semilla para miembros act...   
42      496  Proyectos productivos colectivos e individuale...   
410     927  Sistema de Información del Servicio Público de...   
36     2832  Beneficio de Estímulo Económico de Sometimient...   
268     946  Gratuidad en la matrícula en las Instituciones...   
140    3605                                CIRCULOS SOLIDARIOS   
54     1834                                    CONECTA DIGITAL   
150     725             Núcleo E capítulo Fondo Mujer Emprende   
55     2972        Línea de crédito Creo Economía Popular 2023   
375     449                               Emprendimiento- SENA   
335    2011                                Tu Negocio En Linea   
155    1760                                         Ruta ALDEA   
148    283

In [96]:
recommendations.to_excel('C:/Backup - Archivos de trabajo/DNP/Automatización ArCo/Excel/recomendaciones.xlsx', index=False)